In [11]:
import numpy as np
import pandas as pd
from pathlib import Path
import optuna

from scipy.optimize import minimize
from scipy.optimize import Bounds

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

In [2]:
path_results = Path("../data/results/")

In [3]:
all_oof = dict(
    lightgbm = pd.read_parquet(path_results / "oof_lightgbm_cv1.parquet"),
    xgboost = pd.read_parquet(path_results / "oof_xgboost_cv1.parquet"),
    catboost = pd.read_parquet(path_results / "oof_catboost_cv1.parquet"),
    dt_nn = pd.read_parquet(path_results / "oof_dt-nn_cv1.parquet"),
    mlp = pd.read_parquet(path_results / "oof_nn-mlp_cv1.parquet"),
    gandalf = pd.read_parquet(path_results / "oof_nn-gandalf_cv1.parquet"),
)

combined_df = all_oof["lightgbm"][["utility_agent1_true","fold"]]
for model, df in all_oof.items():
    combined_df[f'utility_agent1_pred_{model}'] = df['utility_agent1_pred']

combined_df = combined_df.dropna(subset=["fold"], ignore_index=True)
combined_df["fold"] = combined_df["fold"].astype(int)
combined_df

,utility_agent1_true,fold,utility_agent1_pred_lightgbm,utility_agent1_pred_xgboost,utility_agent1_pred_catboost,utility_agent1_pred_dt_nn,utility_agent1_pred_mlp,utility_agent1_pred_gandalf
0,-0.466667,2,-0.085834,-0.266970,-0.118676,-0.172795,-0.090974,-0.409328
1,-0.333333,2,0.127977,0.370523,-0.001185,0.256416,-0.066290,-0.128431
2,-0.066667,2,0.231497,0.121204,0.066214,0.149334,-0.116330,-0.016309
3,-0.333333,2,0.032478,0.040438,0.034806,0.138708,-0.080450,-0.154246
4,-0.333333,2,0.179910,0.481094,0.153631,0.228635,0.003619,0.051072
...,...,...,...,...,...,...,...,...
208603,-0.733333,1,-0.452967,-0.456596,-0.496038,-0.497529,-0.173873,-0.324641
208604,0.266667,1,-0.337590,-0.331496,-0.094571,-0.169544,-0.202842,-0.336557
208605,0.666667,1,0.031267,-0.186448,-0.077567,-0.000092,0.151975,-0.053128
208606,0.666667,1,0.010582,-0.272336,-0.026679,0.027810,0.059541,-0.162267


***
## blend test

In [4]:
# Dictionary to store scores for each model
model_scores = {}

# Get all prediction columns
pred_columns = [col for col in combined_df.columns if 'pred' in col]

for model_col in pred_columns:
    scores = list()
    
    for _, df in combined_df.groupby("fold"):
        pred = df[model_col].clip(-1, 1)
        squared_errors = (df['utility_agent1_true'] - pred)**2
        rmse = np.sqrt(squared_errors.mean())
        scores.append(rmse)
        
    model_scores[model_col] = scores
    print(f"\n{model_col}:")
    print("Scores per fold:", [f"{score:.4f}" for score in scores])
    print("Average score: {:.4f}".format(np.mean(scores)))


utility_agent1_pred_lightgbm:
Scores per fold: ['0.4348', '0.4510', '0.4418', '0.4624', '0.4165']
Average score: 0.4413

utility_agent1_pred_xgboost:
Scores per fold: ['0.4450', '0.4582', '0.4527', '0.4468', '0.4102']
Average score: 0.4426

utility_agent1_pred_catboost:
Scores per fold: ['0.4202', '0.4324', '0.4200', '0.4388', '0.4088']
Average score: 0.4240

utility_agent1_pred_dt_nn:
Scores per fold: ['0.4343', '0.4611', '0.4574', '0.4538', '0.4244']
Average score: 0.4462

utility_agent1_pred_mlp:
Scores per fold: ['0.4411', '0.4446', '0.4543', '0.4552', '0.4092']
Average score: 0.4409

utility_agent1_pred_gandalf:
Scores per fold: ['0.4572', '0.4544', '0.4607', '0.4642', '0.4175']
Average score: 0.4508


In [5]:
# Create a linear blend of all predictions (excluding dt_nn)
pred_columns = [
    col for col in combined_df.columns
    if 'pred' in col 
    and 'dt_nn' not in col
]

# Function to calculate RMSE for given weights
def calculate_blend_rmse(weights, pred_cols, df):
    blended_predictions = np.zeros(len(df))
    for i, col in enumerate(pred_cols):
        blended_predictions += weights[i] * df[col].values
        
    scores = []
    for _, fold_df in df.groupby("fold"):
        fold_preds = blended_predictions[fold_df.index].clip(-1, 1)
        squared_errors = (fold_df['utility_agent1_true'] - fold_preds)**2
        rmse = np.sqrt(squared_errors.mean())
        scores.append(rmse)
    return np.mean(scores)

# Initial weights - equal weights
initial_weights = np.ones(len(pred_columns)) / len(pred_columns)

# Constraint: weights sum to 1
def weight_constraint(weights):
    return np.sum(weights) - 1

constraints = {'type': 'eq', 'fun': weight_constraint}
bounds = Bounds(0, 1)  # Each weight between 0 and 1

# # Try different optimization methods
# methods = ['SLSQP', 'trust-constr', 'COBYLA']
# best_score = float('inf')
# best_result = None

# Optimize weights
result = minimize(
    lambda w: calculate_blend_rmse(w, pred_columns, combined_df),
    initial_weights,
    method='SLSQP',
    bounds=bounds,
    constraints=constraints
)

optimal_weights = result.x

# Calculate final blended predictions with optimal weights
blended_predictions = np.zeros(len(combined_df))
for i, col in enumerate(pred_columns):
    blended_predictions += optimal_weights[i] * combined_df[col].values

# Calculate and print final scores
blend_scores = []
for _, df in combined_df.groupby("fold"):
    fold_preds = blended_predictions[df.index].clip(-1, 1)
    squared_errors = (df['utility_agent1_true'] - fold_preds)**2
    rmse = np.sqrt(squared_errors.mean())
    blend_scores.append(rmse)

print("\nOptimized blend:")
print("Optimal weights:")
for col, weight in zip(pred_columns, optimal_weights):
    print(f"{col}: {weight:.4f}")
print("\nScores per fold:", [f"{score:.4f}" for score in blend_scores])
print("Average score: {:.4f}".format(np.mean(blend_scores)))


Optimized blend:
Optimal weights:
utility_agent1_pred_lightgbm: 0.0409
utility_agent1_pred_xgboost: 0.0703
utility_agent1_pred_catboost: 0.4901
utility_agent1_pred_mlp: 0.2440
utility_agent1_pred_gandalf: 0.1548

Scores per fold: ['0.4158', '0.4241', '0.4153', '0.4293', '0.3939']
Average score: 0.4157


In [10]:
# Create a linear blend of all predictions (excluding dt_nn)
pred_columns = [
    col for col in combined_df.columns
    if 'pred' in col 
    and 'dt_nn' not in col
]

# Function to calculate RMSE for given weights
def calculate_blend_rmse(weights, pred_cols, df):
    blended_predictions = np.zeros(len(df))
    for i, col in enumerate(pred_cols):
        blended_predictions += weights[i] * df[col].values
        
    scores = []
    for _, fold_df in df.groupby("fold"):
        fold_preds = blended_predictions[fold_df.index].clip(-1, 1)
        squared_errors = (fold_df['utility_agent1_true'] - fold_preds)**2
        rmse = np.sqrt(squared_errors.mean())
        scores.append(rmse)
    return np.mean(scores)

# Initial weights - equal weights
initial_weights = np.ones(len(pred_columns)) / len(pred_columns)

# Try different optimization methods
methods = ['Nelder-Mead', 'Powell', 'CG', 'BFGS', 'L-BFGS-B', 'TNC', 'COBYLA', 'SLSQP']

for method in methods:
    print("-"*100)
    print(f"Optimizing blend using {method}:")
    try:
        result = minimize(
            lambda w: calculate_blend_rmse(w, pred_columns, combined_df),
            initial_weights,
            method=method
        )
            
        if not result.success:
            print(f"\nOptimizer {method} failed to converge")
            continue
            
        optimal_weights = result.x

        # Calculate final blended predictions with optimal weights
        blended_predictions = np.zeros(len(combined_df))
        for i, col in enumerate(pred_columns):
            blended_predictions += optimal_weights[i] * combined_df[col].values

        # Calculate and print final scores
        blend_scores = []
        for _, df in combined_df.groupby("fold"):
            fold_preds = blended_predictions[df.index].clip(-1, 1)
            squared_errors = (df['utility_agent1_true'] - fold_preds)**2
            rmse = np.sqrt(squared_errors.mean())
            blend_scores.append(rmse)

        print(f"\nOptimized blend using {method}:")
        print("Optimal weights:")
        for col, weight in zip(pred_columns, optimal_weights):
            print(f"{col}: {weight:.4f}")
        print("\nScores per fold:", [f"{score:.4f}" for score in blend_scores])
        print("Average score: {:.4f}".format(np.mean(blend_scores)))
        
    except Exception as e:
        print(f"\nOptimizer {method} failed with error: {str(e)}")

----------------------------------------------------------------------------------------------------
Optimizing blend using Nelder-Mead:

Optimized blend using Nelder-Mead:
Optimal weights:
utility_agent1_pred_lightgbm: 0.0381
utility_agent1_pred_xgboost: 0.0786
utility_agent1_pred_catboost: 0.4959
utility_agent1_pred_mlp: 0.2457
utility_agent1_pred_gandalf: 0.1528

Scores per fold: ['0.4158', '0.4242', '0.4152', '0.4294', '0.3938']
Average score: 0.4157
----------------------------------------------------------------------------------------------------
Optimizing blend using Powell:

Optimized blend using Powell:
Optimal weights:
utility_agent1_pred_lightgbm: 0.0402
utility_agent1_pred_xgboost: 0.0746
utility_agent1_pred_catboost: 0.4983
utility_agent1_pred_mlp: 0.2421
utility_agent1_pred_gandalf: 0.1565

Scores per fold: ['0.4158', '0.4241', '0.4151', '0.4295', '0.3938']
Average score: 0.4157
--------------------------------------------------------------------------------------------

In [12]:
# optimize using optuna

def objective(trial):
    # Generate weights that sum to 1 using softmax
    raw_weights = [trial.suggest_float(f'w_{i}', -10, 10) for i in range(len(pred_columns))]
    weights = np.exp(raw_weights) / np.sum(np.exp(raw_weights))
    
    return calculate_blend_rmse(weights, pred_columns, combined_df)

# Create and run study
# First 1000 trials with QMC sampler
study_qmc = optuna.create_study(
    study_name="blend_optuna",
    direction='minimize',
    load_if_exists=False,
    sampler=optuna.samplers.QMCSampler()
)
study_qmc.optimize(objective, n_trials=1000)

# Next 1000 trials with TPE sampler, starting from QMC results
study_tpe = optuna.create_study(
    study_name="blend_optuna",
    direction='minimize',
    load_if_exists=True,
    sampler=optuna.samplers.TPESampler(
        n_startup_trials=1,    # Increase random sampling at start
        n_ei_candidates=100,   # Consider more candidates
        multivariate=True,     # Enable multivariate sampling
        constant_liar=True     # Help with parallel optimization
    )
)

# Transfer trials from QMC study to TPE study
for trial in study_qmc.trials:
    if trial.state == optuna.trial.TrialState.COMPLETE:
        study_tpe.add_trial(trial)

# Continue optimization with TPE sampler
study_tpe.optimize(objective, n_trials=1000)




# Get best weights using softmax
raw_weights = [study_tpe.best_params[f'w_{i}'] for i in range(len(pred_columns))]
optimal_weights = np.exp(raw_weights) / np.sum(np.exp(raw_weights))

# Calculate final blended predictions with optimal weights
blended_predictions = np.zeros(len(combined_df))
for i, col in enumerate(pred_columns):
    blended_predictions += optimal_weights[i] * combined_df[col].values

# Calculate and print final scores
blend_scores = []
for _, df in combined_df.groupby("fold"):
    fold_preds = blended_predictions[df.index].clip(-1, 1)
    squared_errors = (df['utility_agent1_true'] - fold_preds)**2
    rmse = np.sqrt(squared_errors.mean())
    blend_scores.append(rmse)

print("\nOptimized blend using Optuna:")
print("Optimal weights:")
for col, weight in zip(pred_columns, optimal_weights):
    print(f"{col}: {weight:.4f}")
print("\nScores per fold:", [f"{score:.4f}" for score in blend_scores])
print("Average score: {:.4f}".format(np.mean(blend_scores)))


[I 2024-11-25 16:31:35,605] A new study created in memory with name: no-name-1643a6f7-93ca-4101-8571-8f24133415a9
[I 2024-11-25 16:31:35,626] Trial 0 finished with value: 0.4216054618385253 and parameters: {'w_0': -0.1903993469266556, 'w_1': 3.2864912878740054, 'w_2': 8.278173907033203, 'w_3': 5.690213050708335, 'w_4': -8.607124308073605}. Best is trial 0 with value: 0.4216054618385253.
[I 2024-11-25 16:31:35,643] Trial 1 finished with value: 0.41906474787879366 and parameters: {'w_0': 1.0530218893847945, 'w_1': -1.4448003728798042, 'w_2': 7.70990043292263, 'w_3': 7.736034396284651, 'w_4': -0.5904815089878888}. Best is trial 1 with value: 0.41906474787879366.
[I 2024-11-25 16:31:35,654] Trial 2 finished with value: 0.44028862280629566 and parameters: {'w_0': 3.031612284087643, 'w_1': -4.300427940759919, 'w_2': -6.188817642053093, 'w_3': -1.9375115297297167, 'w_4': -1.4661979448072309}. Best is trial 1 with value: 0.41906474787879366.
[I 2024-11-25 16:31:35,664] Trial 3 finished with va


Optimized blend using Optuna:
Optimal weights:
utility_agent1_pred_lightgbm: 0.0069
utility_agent1_pred_xgboost: 0.0707
utility_agent1_pred_catboost: 0.5127
utility_agent1_pred_mlp: 0.2457
utility_agent1_pred_gandalf: 0.1640

Scores per fold: ['0.4160', '0.4241', '0.4153', '0.4291', '0.3940']
Average score: 0.4157


In [9]:
from sklearn.linear_model import LinearRegression, Ridge, RidgeCV, LassoCV, ElasticNetCV
 
# Create a linear blend of all predictions (excluding dt_nn) using sklearn LinearRegression
pred_columns = [
    col for col in combined_df.columns 
    if 'pred' in col
    and 'dt_nn' not in col
]

# Split data by fold and fit/predict on each fold
blend_scores = []
blended_predictions = np.zeros(len(combined_df))

for fold in combined_df['fold'].unique():
    # Split into train/test for this fold
    train_mask = combined_df['fold'] != fold
    test_mask = combined_df['fold'] == fold
    
    X_train = combined_df.loc[train_mask, pred_columns]
    y_train = combined_df.loc[train_mask, 'utility_agent1_true']
    X_test = combined_df.loc[test_mask, pred_columns]
    y_test = combined_df.loc[test_mask, 'utility_agent1_true']
    
    # Fit linear regression
    lr = LinearRegression(positive=True)  # Enforce positive weights
    # lr = Ridge(positive=True)
    # lr = RidgeCV()
    # lr = LassoCV()
    # lr = ElasticNetCV()
    lr.fit(X_train, y_train)
    
    # Make predictions
    fold_preds = lr.predict(X_test).clip(-1, 1)
    blended_predictions[test_mask] = fold_preds
    
    # Calculate score for this fold
    squared_errors = (y_test - fold_preds)**2
    rmse = np.sqrt(squared_errors.mean())
    blend_scores.append(rmse)

# Get final weights by fitting on all data
# lr_final = LinearRegression(positive=True)
# lr_final = RidgeCV()
# lr_final = LassoCV()
# lr_final = ElasticNetCV()
# lr_final = LinearSVR()

# lr_final.fit(combined_df[pred_columns], combined_df['utility_agent1_true'])
# optimal_weights = lr_final.coef_
# # Normalize weights to sum to 1
# optimal_weights = optimal_weights / np.sum(optimal_weights)

# print("\nOptimized blend:")
# print("Optimal weights:")
# for col, weight in zip(pred_columns, optimal_weights):
#     print(f"{col}: {weight:.4f}")

print("\nScores per fold:", [f"{score:.4f}" for score in blend_scores])
print("Average score: {:.4f}".format(np.mean(blend_scores)))


Scores per fold: ['0.4246', '0.4168', '0.4166', '0.3949', '0.4304']
Average score: 0.4167


***
## post-proc test

In [9]:
oof = pd.read_csv(path_results / "oof_catboost_cv1.csv")
oof = oof.dropna(subset=["fold"], ignore_index=True)
oof["fold"] = oof["fold"].astype(int)
oof

,utility_agent1_true,utility_agent1_pred,fold
0,-0.466667,-0.118676,2
1,-0.333333,-0.001185,2
2,-0.066667,0.066214,2
3,-0.333333,0.034806,2
4,-0.333333,0.153631,2
...,...,...,...
208603,-0.733333,-0.496038,1
208604,0.266667,-0.094571,1
208605,0.666667,-0.077567,1
208606,0.666667,-0.026679,1


In [10]:
def compute_cv_score(oof_df):
    scores = list()
    for _, df in oof_df.groupby("fold"):
        pred = df["utility_agent1_pred"].clip(-1, 1)
        squared_errors = (df['utility_agent1_true'] - pred)**2
        rmse = np.sqrt(squared_errors.mean())
        scores.append(rmse)
    return np.mean(scores)

In [11]:
compute_cv_score(oof)

0.42404969692411615

In [12]:
_oof = oof.copy()

w = 1.1
shift=0
p_min = -0.985
p_max = 0.985

_oof["utility_agent1_pred"] = (_oof["utility_agent1_pred"] * w + shift).clip(p_min, p_max)

compute_cv_score(_oof)

0.4252171849796663